<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/jcc/03-BERT-wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JCC: BERT con ajuste de hiperparámetros

En esta notebook se presentan los experimentos sobre la estrategia de representación y técnica de aprendizaje *avanzada* utilizada para las JCC de  la Universidad Nacional de La Plata.

Para ello vamos a tomar el texto de la consulta de los correos y aplicar BERT con el modelo pre-entrenado en español (BETO - Departamento de Computación de la Universidad de Chile).


## Instalación de librerías

Se instalan dos librerías que no están en el entorno Colab:
- *simpletransformers* (para entrenar Bert),
- requests (para consumir funciones propias de Github),
- wget (para la descarga de archivos),
- wandb (para la optimización de hiperparámetros).

In [ ]:
!pip install simpletransformers
!pip install requests
!pip install wget
!pip install wandb

### Funciones útiles

Se cargan funciones útiles desde el repo https://github.com/jumafernandez/clasificacion_correos para la carga y balanceo del dataset.

In [ ]:
import requests

# Se hace el request del raw del script python
url = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/scripts/jcc/funciones_dataset.py'
r = requests.get(url)

# Se guarda en el working directory
with open('funciones_dataset.py', 'w') as f:
    f.write(r.text)

# Se importan las funciones a utilizar
from funciones_dataset import get_clases, cargar_dataset, separar_x_y_rna

También se carga la función para preprocesar el texto que se usó en los otros modelos desde el repo: https://github.com/jumafernandez/clasificacion_correos.

In [ ]:

import requests

# Se hace el request del raw del script python
url = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/scripts/jcc/funciones_preprocesamiento.py'
r = requests.get(url)

# Se guarda en el working directory
with open('funciones_preprocesamiento.py', 'w') as f:
    f.write(r.text)

# Se importan las funciones a utilizar
from funciones_preprocesamiento import preprocesar_correos_bert

## Carga del dataset con los correos

Se cargan los datos y se realiza el balanceo de clases:

In [ ]:
from funciones_dataset import get_clases, cargar_dataset
from os import path
import warnings
warnings.filterwarnings("ignore")

# Cantidad de clases
CANTIDAD_CLASES = 4

# Constantes con los datos
DS_DIR = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/consolidado_jcc/'
TRAIN_FILE = 'correos-train-80.csv'
TEST_FILE = 'correos-test-20.csv'

# Chequeo sobre si los archivos están en el working directory
download_files = not(path.exists(TRAIN_FILE))

etiquetas = get_clases()
train_df, test_df, etiquetas = cargar_dataset(DS_DIR, TRAIN_FILE, TEST_FILE, download_files, 'clase', etiquetas, CANTIDAD_CLASES, 'Otras Consultas')


Se inicia descarga de los datasets.

El conjunto de entrenamiento tiene la dimensión: (800, 24)
El conjunto de testeo tiene la dimensión: (200, 24)


### Pre-procesamiento de los datos

Se preparan los datos para el entrenamiento de BERT:

In [ ]:
train_df = train_df[['Consulta', 'clase']]
train_df.columns = ['text', 'labels']
test_df = test_df[['Consulta', 'clase']]
test_df.columns = ['text', 'labels']

# Cambio los integers por las etiquetas
train_df.labels = etiquetas[train_df.labels]
test_df.labels = etiquetas[test_df.labels]

# Las vuelvo a pasar a números 0-N para evitar conflictos con simpletransformers
# Este paso está fijo para estos experimentos
dict_clases_id = {'Otras Consultas': 0,
                            'Ingreso a la Universidad': 1,
                            'Boleto Universitario': 2,
                            'Requisitos de Ingreso': 3}

train_df['labels'].replace(dict_clases_id, inplace=True)
test_df['labels'].replace(dict_clases_id, inplace=True)

# Muestro salida por consola
print('Existen {} clases: {}.'.format(len(train_df.labels.unique()), train_df.labels.unique()))

Existen 4 clases: [0 1 2 3].


## Elección de un modelo monolenguaje pre-entrenado

La librería *simpletransformers* se basa en la librería *Transformers* de HuggingFace. Esto permite utilizar todos los modelos pre-entrenados disponibles en la [Transformers library](https://huggingface.co/transformers/pretrained_models.html) que son provistos por toda la comunidad de desarrolladores. Para ver cuales son los modelos disponibles, se puede ingresar a [https://huggingface.co/models](https://huggingface.co/models).

En nuestro caso, vamos a utilizar el modelo `dccuchile/bert-base-spanish-wwm-cased`. Este modelo está pre-entrenado por un equipo de investigadores del Departamento de Computación de la Universidad de Chile.

### Optimización de hiperparámetros

Se utiliza Sweep de wandb para la optimización de hiperparámetros.

Para más información:
https://simpletransformers.ai/docs/tips-and-tricks/#hyperparameter-optimization

In [ ]:
import wandb

sweep_config = {
    "method": "bayes",  # grid, random
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"values": [2, 3, 4]},
        "learning_rate": {"min": 4e-5, "max": 4e-4},
    },
}

sweep_id = wandb.sweep(sweep_config, project="Bert_JCC2021")

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 5ocb1k9j
Sweep URL: https://wandb.ai/jumafernandez/Bert_JCC2021/sweeps/5ocb1k9j


Trackeo del logg de aprendizaje del Modelo:

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

### Definición del Modelo para Clasificación

Se carga el modelo pre-entrenado BETO con la respectiva definición de hiperparámetros para el entrenamiento:

In [ ]:
from simpletransformers.classification import ClassificationModel

# Hiperparámetros
train_args = {
        'overwrite_output_dir': True,
        'reprocess_input_data': True,
        'evaluate_during_training': True,
        'fp16': True,
        'do_lower_case': True,
        'use_early_stopping': True,
        'manual_seed': 4,
        'use_multiprocessing': True,
        'train_batch_size': 16,
        'eval_batch_size': 8,
        'wandb_project': 'Bert_JCC2021',
        }

## Entrenamiento

Se entrena el modelo con el dataset de train en función de los hiperparámetros:

In [ ]:
def train():
    # Se inicializa wandb
    wandb.init()

    # Creamos el ClassificationModel
    model = ClassificationModel(
        model_type='bert', 
    #    model_name='bert-base-multilingual-cased',
        model_name='dccuchile/bert-base-spanish-wwm-cased',
        num_labels=CANTIDAD_CLASES,
        use_cuda=False,
        args=train_args
    )

    # Se entrena el modelo
    model.train_model(train_df, eval_df=test_df)

    # Se evalúa el modelo
    model.eval_model(test_df)

    # Se sincroniza el run
    wandb.join()

wandb.agent(sweep_id, train)

INFO:wandb.agents.pyagent:Starting sweep agent: entity=None, project=None, count=None
wandb: Agent Starting Run: p01pmrsz with config:
wandb: 	learning_rate: 0.0003325194503654975
wandb: 	num_train_epochs: 3
wandb: Currently logged in as: jumafernandez (use `wandb login --relogin` to force relogin)


INFO:filelock:Lock 139777078333776 acquired on /root/.cache/huggingface/transformers/cb7cedb04246e225d56ba26d207f1d1809b31a9bbe9b63103371d835c6ac0502.f4e4777229bac528fa2a8d4833e2ef53624e985ebde0fd527064a5cc7c50832b.lock


Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

INFO:filelock:Lock 139777078333776 released on /root/.cache/huggingface/transformers/cb7cedb04246e225d56ba26d207f1d1809b31a9bbe9b63103371d835c6ac0502.f4e4777229bac528fa2a8d4833e2ef53624e985ebde0fd527064a5cc7c50832b.lock
INFO:filelock:Lock 139777044493200 acquired on /root/.cache/huggingface/transformers/52382cbe7c1587c6b588daa81eaf247c5e2ad073d42b52192a8cd4202e7429b6.a88ccd19b1f271e63b6a901510804e6c0318089355c471334fe8b71b316a30ab.lock


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

  0%|          | 0/800 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_4_2


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/50 [00:00<?, ?it/s]